<a href="https://colab.research.google.com/github/Tekleab15/Regularized_Auto_Encoder/blob/main/RAE_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regularized Autoencoder Implementation
This notebook demonstrates the implementation of a Regularized Autoencoder (RAE) using backpropagation, inspired by the methodologies described in "The Neural Coding Framework for Learning Generative Models".



**Dataset Loading and Preprocessing: **all the four datasets used in the paper (MNIST, KMNIST, FMNIST, CalTech101)

# 1. Loading and Preprocessing MNIST dataset

In [26]:
# loading mnist dataset from the open online keras datasets
from tensorflow.keras.datasets import mnist

# load the dataset as it's splitted into a training and testing sets
(x_train_mnist, y_train_mnist), (x_test_mnist, y_test_mnist) = mnist.load_data()

# Normalizing(setting between 0 and 1) and reshape the datasets loaded
x_train_mnist = x_train_mnist.astype('float32') / 255.
x_test_mnist = x_test_mnist.astype('float32') / 255.
x_train_mnist = x_train_mnist.reshape((x_train_mnist.shape[0], -1))
x_test_mnist = x_test_mnist.reshape((x_test_mnist.shape[0], -1))

In [28]:
# Assuring the dataset is reshaped successfully
print("The shape of the dataset is ", x_train_mnist.shape)

The shape of the dataset is  (60000, 784)


# 2. Loading and Preprocessing KMNIST dataset

In [30]:
# Unlike MNIST datasets KMNIST datasets should be loaded in a little bit different way
import tensorflow_datasets as tfds
import numpy as np
(x_train_kmnist, y_test_kmnist), ds_info = tfds.load(
    'kmnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)
def convert_to_numpy(ds):
    images, labels = [], []
    for img, lbl in tfds.as_numpy(ds):
        images.append(img)
        labels.append(lbl)
    return np.array(images), np.array(labels)

# Convert datasets to numpy arrays
x_train_kmnist, y_train_kmnist = convert_to_numpy(x_train_kmnist)
x_test_kmnist, y_test_kmnist = convert_to_numpy(y_test_kmnist)

# Normalize and Reshape the images as we did for MNIST
x_train_kmnist = x_train_kmnist.astype('float32') / 255.
x_test_kmnist = x_test_kmnist.astype('float32') / 255.

x_train_kmnist = x_train_kmnist.reshape((x_train_kmnist.shape[0], 28 * 28))
x_test_kmnist = x_test_kmnist.reshape((x_test_kmnist.shape[0], 28 * 28))

In [29]:
# Assuring the dataset is reshaped successfully
print(f'KMNIST train shape: {x_train_kmnist.shape}, KMNIST test shape: {x_test_kmnist.shape}')

KMNIST train shape: (60000, 28, 28, 1), KMNIST test shape: (10000, 28, 28, 1)


# 3. Loading and Preprocessing the FMNIST dataset

In [34]:
# Loading the FMNIST dataset
(x_train_fmnist, y_test_fmnist), ds_info = tfds.load(
    'fashion_mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)

x_train_fmnist, y_train_fmnist = convert_to_numpy(x_train_fmnist)
x_test_fmnist, y_test_fmnist = convert_to_numpy(y_test_fmnist)

# Normalize and reshape the images
x_train_fmnist = x_train_fmnist.astype('float32') / 255.
x_test_fmnist = x_test_fmnist.astype('float32') / 255.

x_train_fmnist = x_train_fmnist.reshape((x_train_fmnist.shape[0], 28 * 28))
x_test_fmnist = x_test_fmnist.reshape((x_test_fmnist.shape[0], 28 * 28))
print("The dataset size and shape: ", x_train_fmnist.shape)

The dataset size and shape:  (60000, 784)


# 4. Loading and Preprocessing CalTech101